<a href="https://colab.research.google.com/github/MajstorKatastrofe/AI_NM2/blob/main/NM_Domaci2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim


In [ ]:
# Define transformations to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Load the Fashion-MNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                            download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                           download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


In [ ]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')


In [ ]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Initialize the model, loss function, and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# Train the model
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


[1,   100] loss: 2.302
[1,   200] loss: 2.298
[1,   300] loss: 2.290
[1,   400] loss: 2.278
[1,   500] loss: 2.253
[1,   600] loss: 2.181
[1,   700] loss: 1.897
[1,   800] loss: 1.362
[1,   900] loss: 1.042
[2,   100] loss: 0.862
[2,   200] loss: 0.809
[2,   300] loss: 0.761
[2,   400] loss: 0.728
[2,   500] loss: 0.696
[2,   600] loss: 0.685
[2,   700] loss: 0.662
[2,   800] loss: 0.697
[2,   900] loss: 0.656
[3,   100] loss: 0.655
[3,   200] loss: 0.637
[3,   300] loss: 0.612
[3,   400] loss: 0.602
[3,   500] loss: 0.598
[3,   600] loss: 0.590
[3,   700] loss: 0.580
[3,   800] loss: 0.585
[3,   900] loss: 0.561
[4,   100] loss: 0.568
[4,   200] loss: 0.536
[4,   300] loss: 0.554
[4,   400] loss: 0.555
[4,   500] loss: 0.533
[4,   600] loss: 0.543
[4,   700] loss: 0.534
[4,   800] loss: 0.521
[4,   900] loss: 0.520
[5,   100] loss: 0.514
[5,   200] loss: 0.514
[5,   300] loss: 0.494
[5,   400] loss: 0.511
[5,   500] loss: 0.511
[5,   600] loss: 0.490
[5,   700] loss: 0.472
[5,   800] 

In [ ]:
# Set the model to evaluation mode
net.eval()

# Initialize variables to keep track of validation loss and accuracy
val_loss = 0.0
correct = 0
total = 0

# Load the Fashion-MNIST validation dataset
val_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Iterate over the validation dataset
with torch.no_grad():
    for data in val_dataloader:
        images, labels = data

        # Forward pass
        outputs = net(images)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Accumulate the validation loss
        val_loss += loss.item()

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        # Update the total number of samples
        total += labels.size(0)

        # Count the number of correct predictions
        correct += (predicted == labels).sum().item()

# Calculate the average validation loss
avg_val_loss = val_loss / len(val_dataloader)

# Calculate the validation accuracy
accuracy = correct / total

# Print the validation loss and accuracy
print('Validation Loss: {:.4f}'.format(avg_val_loss))
print('Validation Accuracy: {:.2f}%'.format(100 * accuracy))


Validation Loss: 0.4123
Validation Accuracy: 84.97%


In [ ]:
# Save the trained model parameters to a file
torch.save(net.state_dict(), 'fashion_mnist_cnn.pth')


In [ ]:
# Load the trained model parameters
net.load_state_dict(torch.load('fashion_mnist_cnn.pth'))


<All keys matched successfully>

In [ ]:
# Test the model on one photo from each category
with torch.no_grad():
    category_predictions = {}
    for images, labels in testloader:
        for image, label in zip(images, labels):
            category = classes[label.item()]
            if category not in category_predictions:
                # Get the predicted label
                outputs = net(image.unsqueeze(0))
                _, predicted = torch.max(outputs, 1)
                predicted_label = classes[predicted.item()]

                # Store the predicted label for this category
                category_predictions[category] = predicted_label

                # Print the result
                print(f"Category: {category}, Predicted Label: {predicted_label}")

            # Stop if we have one prediction for each category
            if len(category_predictions) == len(classes):
                break

        # Stop if we have one prediction for each category
        if len(category_predictions) == len(classes):
            break


Category: Ankle boot, Predicted Label: Ankle boot
Category: Pullover, Predicted Label: Pullover
Category: Trouser, Predicted Label: Trouser
Category: Shirt, Predicted Label: Shirt
Category: Coat, Predicted Label: Coat
Category: Sandal, Predicted Label: Sandal
Category: Sneaker, Predicted Label: Sneaker
Category: Dress, Predicted Label: Dress
Category: Bag, Predicted Label: Bag
Category: T-shirt/top, Predicted Label: T-shirt/top
